In [ ]:
import networkx as nx
import pandas as pd
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import make_spd_matrix

# Small example

In [ ]:
n_nodes = 30
seed = 41
infection_length = 7
p_infectiousness = 0.2
n_samples = 2

In [ ]:
G = nx.generators.random_graphs.fast_gnp_random_graph(n_nodes, p = 0.3)

In [ ]:
G = nx.generators.powerlaw_cluster_graph(20, 5, 0.2)

In [ ]:
nx.draw_networkx(G, pos=nx.spring_layout(G))

Infect a random subset of nodes

In [ ]:
np.random.seed(seed + 2)
initial_infections = np.random.binomial(1, 0.1, n_nodes)

nx.set_node_attributes(G, dict(zip(np.arange(n_nodes), initial_infections)), name = "infected")
nx.set_node_attributes(G, dict(zip(np.arange(n_nodes), np.zeros(n_nodes))), name = "recovered")
nx.set_node_attributes(G, dict(zip(np.arange(n_nodes), np.where(initial_infections == 0, 1, 0))), name = "susceptible")
nx.set_node_attributes(G, dict(zip(np.arange(n_nodes), np.zeros(n_nodes))), name = "quarantined")

In [ ]:
np.where(initial_infections == 0, 1, 0)

In [ ]:
nx.draw_networkx(G, pos=nx.spring_layout(G, seed=seed), node_color = list(nx.get_node_attributes(G, 'infected').values()), )

Standard random sample

In [ ]:
from time import sleep

In [ ]:
def progress_disease(G):
    
    infected_nodes = [x for x,y in G.nodes(data=True) if (y['infected']>0)]
    for idx in infected_nodes:
        if G.nodes[idx]['infected'] >= infection_length:
            G.nodes[idx]['infected'] = 0
            G.nodes[idx]['recovered'] = 1
        else:
            G.nodes[idx]['infected'] += 1

        if G.nodes[idx]['quarantined'] == 0:
            #print("infecting others")
            neighbours = [n for n in G[idx].keys() if G.nodes[n]['susceptible'] == 1]            
            for neighbour in neighbours:
                G.nodes[neighbour]['susceptible'] = 0
                G.nodes[neighbour]['infected'] = np.random.binomial(1, p_infectiousness)
    return G

In [ ]:
def sample_and_quarantine(G, idx_to_sample):
    for idx in idx_to_sample:
        if G.nodes[idx]['infected'] > 0:
            G.nodes[idx]['quarantined'] = 1
            
    return G

In [ ]:
def random_sample(G, seed = 42):
    np.random.seed(seed)
    n_infected = len([x for x,y in G.nodes(data=True) if y['infected']>0])
    while (n_infected > 0) :

        # randomly sample, if positive, quarantine
        idx_to_sample = np.random.choice(np.arange(n_nodes), n_samples, replace=False)
        G = sample_and_quarantine(G, idx_to_sample)

        # progess the disease along the network
        G = progress_disease(G)

        n_infected = len([x for x,y in G.nodes(data=True) if y['infected']>0])
        
    return len([x for x,y in G.nodes(data=True) if y['recovered']])

In [ ]:
def random_neigbour_sample(G, seed = 42):
    np.random.seed(seed)
    n_infected = len([x for x,y in G.nodes(data=True) if y['infected']>0])
    while (n_infected > 0) :
    
        # randomly sample, if positive, quarantine
        idx_to_sample = np.random.choice(np.arange(n_nodes), n_samples, replace=False)
        # Get it's neighbour instead
        idx_to_sample = [np.random.choice(list(G[idx].keys())) for idx in idx_to_sample]
        
        #while len(set_of_all_neighbours) < n_samples:
        #    set_of_all_neighbours = set_of_all_neighbours.union(np.random.choice(idx_to_sample, n_samples - len(set_of_all_neighbours), replace = False))
            
        #idx_to_sample = np.random.choice(set_of_all_neighbours, n_samples, replace=False)
        #print(set_of_all_neighbours)
        
        G = sample_and_quarantine(G, idx_to_sample)

        # progess the disease along the network
        G = progress_disease(G)

        n_infected = len([x for x,y in G.nodes(data=True) if y['infected']>0])
    
    return len([x for x,y in G.nodes(data=True) if y['recovered']])

In [ ]:
n_iters = 1000
results = []

n_nodes = 1000
seed = 50
infection_length = 14
p_infectiousness = 0.5
n_samples = 15


for i in range(n_iters):
    
    #G = nx.generators.random_graphs.fast_gnp_random_graph(n_nodes, p = 0.3)
    G = nx.generators.powerlaw_cluster_graph(n_nodes, 1, 0)
    
    np.random.seed(seed + i)
    initial_infections = np.random.binomial(1, 0.1, n_nodes)

    nx.set_node_attributes(G, dict(zip(np.arange(n_nodes), initial_infections)), name = "infected")
    nx.set_node_attributes(G, dict(zip(np.arange(n_nodes), np.zeros(n_nodes))), name = "recovered")
    nx.set_node_attributes(G, dict(zip(np.arange(n_nodes), np.where(initial_infections == 0, 1, 0))), name = "susceptible")
    nx.set_node_attributes(G, dict(zip(np.arange(n_nodes), np.zeros(n_nodes))), name = "quarantined")
    
    H = G.copy()
    
    results.append({'neighbour': random_neigbour_sample(H, seed + i), 'random': random_sample(G, seed + i)})
    

In [ ]:
df = pd.DataFrame(results)
df.mean(), df.std()

In [ ]:
plt.plot(df['neighbour'], df['random'], ".")
plt.plot([100, 400], [100, 400])

# Test to get the most people

In [ ]:
import pymc3 as pm
import theano.tensor as tt
import theano as tt
import json
import pandas as pd
import plotly.express as px


In [ ]:
!pwd

In [ ]:
with open("../../../Documents/IDInsight/Covid/delhi_covid_analytics/data/00_raw/Daily_analytical_report/supporting data/Delhi_wards.json") as g:
    wards = json.load(g)
    
df = pd.read_csv("prevelance_data.csv")
df['actual_prevalence'] = df['proj']

In [ ]:
fig = px.choropleth(df, geojson=wards, locations="id", featureidkey="properties.IS_WC",
                        projection="mercator",
                        color='actual_prevalence', 
                        hover_data = {'actual_prevalence': ':.2f', 'id':False},
                        )
fig.update(layout_coloraxis_showscale=False)
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0}, showlegend=False)
fig.write_html("../../../sidravi1.github.io/_includes/blog_contents/actuals_delhi.html")
fig.show()

In [ ]:
df['trials'] = np.random.randint(100, 1000, size= df.actual_prevalence.shape[0])
df['successes'] = st.binom(df.trials, df.actual_prevalence).rvs()
df['success_rate'] = df['successes'] / df['trials']

In [ ]:
fig = px.choropleth(df, geojson=wards, locations="id", featureidkey="properties.IS_WC",
                        projection="mercator",
                        color='success_rate', 
                        hover_data = {'actual_prevalence': ':.2f','success_rate': ':.2f', 'id':False},
                        )
fig.update(layout_coloraxis_showscale=False)
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0}, showlegend=False)
fig.write_html("../../../sidravi1.github.io/_includes/blog_contents/raw_rates_delhi.html")
fig.show()

In [ ]:
X = df[['lat', 'lon']].values

X_std = (X - X.mean(axis = 0)) / X.std(axis = 0)
y = df['successes'].values
n = df['trials'].values

In [ ]:
with pm.Model() as gp_field:
    
    rho_x1 = pm.Exponential("rho_x1", lam=5)
    eta_x1 = pm.Exponential("eta_x1", lam=2)

    rho_x2 = pm.Exponential("rho_x2", lam=5)
    eta_x2 = pm.Exponential("eta_x2", lam=2)
    
    K_x1 = eta_x1**2 * pm.gp.cov.ExpQuad(1, ls=rho_x1)
    K_x2 = eta_x2**2 * pm.gp.cov.ExpQuad(1, ls=rho_x2)
    
    gp_x1 = pm.gp.Latent(cov_func=K_x1)
    gp_x2 = pm.gp.Latent(cov_func=K_x2)
    
    f_x1 = gp_x1.prior("f_x1", X=X_std[:,0][:, None])
    f_x2 = gp_x2.prior("f_x2", X=X_std[:,1][:, None])
    
    probs = pm.Deterministic('π', pm.invlogit(f_x1 + f_x2))
    
    obs = pm.Binomial('positive_cases', p = probs, n = n, observed = y.squeeze())
    

In [ ]:
trace = pm.sample(model = gp_field, cores = 1, chains = 1, tune = 1000)

In [ ]:
trace['π'].mean(axis = 0)

In [ ]:
trace['π'].std(axis = 0)

In [ ]:
df['smooth_success_rate'] = trace['π'].mean(axis = 0)

fig = px.choropleth(df, geojson=wards, locations="id", featureidkey="properties.IS_WC",
                        projection="mercator",
                        color='smooth_success_rate', 
                        hover_data = {'actual_prevalence': ':.3f','smooth_success_rate': ':.3f', 'id':False},
                        )
fig.update(layout_coloraxis_showscale=False)
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0}, showlegend=False)
fig.write_html("./fitted_smooth_delhi.html")
fig.show()
